
1. [교차 검증](#교차-검증)
1. [랜덤 서치](#랜덤-서치)
1. [하이퍼파라미터 튜닝](#--하이퍼파라미터-튜닝)

# 교차 검증과 그리드 서치

## - 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 나누어 ***검증 세트*** 라고 불리는 세트를 생성해 사용한다.

앞서 전체 와인 데이터중 20%를 테스트 세트로 만들고 80%를 훈련 세트로 만들었다. 이 *<u>훈련 세트중에서 다시 20%를 떼어 내어 검증 세트로 만든다</u>*.

<img src='https://cdn.imweb.me/upload/S202105076e626e4618b27/0f0c2df8f341c.png' width=600>

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다. 그다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다. 그리고 마지막에 테스트 세트에서 최종 점수를 평가한다.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from regex import splititer

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head(3)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0


In [83]:
wine['alcohol'].min(), wine['alcohol'].max()

(8.0, 14.9)

In [84]:
wine['sugar'].min(), wine['sugar'].max()

(0.6, 65.8)

In [85]:
wine['pH'].min(), wine['pH'].max()

(2.72, 4.01)

### 훈련/검증 세트 분리

In [4]:
X = wine.drop(['class'], axis=1).values
y = wine['class'].values

학습 데이터 분할 

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [6]:
print(X_train.shape, X_test.shape)

(5197, 3) (1300, 3)


이 다음에 훈련세트 X_train과 y_train 으로 검증 훈련 세트 y_val, sub_test 과 검증 테스트 세트 y_val,val_target을 만든다. 
> 여기에서도 test_size 매개변수를 0.2로 지정하여 X_train 의 약 20%를 val_input으로 만든다.

In [7]:
sub_train, val_train, sub_target, val_target = train_test_split(X_train, y_train, test_size=0.2, random_state=50)
print(sub_train.shape, val_train.shape)

(4157, 3) (1040, 3)


원래 5197개 였던 훈련 세트가 4157개로 줄고 검증 세트는 1040개가 되었다. 이제 모델을 만들고 평가해 보자.

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=50)

dt_clf.fit(sub_train, sub_target)

print("훈련세트: ",dt_clf.score(sub_train, sub_target))
print("검증세트 :",dt_clf.score(val_train, val_target))
print("테스트세트 :",dt_clf.score(X_test, y_test))

훈련세트:  0.9980755352417608
검증세트 : 0.8596153846153847
테스트세트 : 0.8623076923076923


classification report 로 전체 점수 확인

In [9]:
from sklearn.metrics import classification_report

pred = dt_clf.predict(sub_train)
print(classification_report(sub_target, pred))

pred2 = dt_clf.predict(val_train)
print(classification_report(val_target, pred2))

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      1019
         1.0       1.00      1.00      1.00      3138

    accuracy                           1.00      4157
   macro avg       1.00      1.00      1.00      4157
weighted avg       1.00      1.00      1.00      4157

              precision    recall  f1-score   support

         0.0       0.74      0.72      0.73       273
         1.0       0.90      0.91      0.91       767

    accuracy                           0.86      1040
   macro avg       0.82      0.82      0.82      1040
weighted avg       0.86      0.86      0.86      1040



이 모델은 훈련 세트에 과대적합되어 있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. 그 전에 검증 세트에 대해 더 알아보자.

## - 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것이다. 이럴 때 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다. 이렇게 하면 데이터의 80 ~ 90% 까지 훈련에 사용할 수 있다. 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.



사이킷런에는 cross_validate() 라는 교차 검증 함수가 있다. 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달한다. 그다음 앞에서처럼 직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 cross_validat() 함수에 전달한다.

```python
def cross_validate(
    estimator: BaseEstimator,
    X: MatrixLike,
    y: MatrixLike | ArrayLike | None = None,
    *,
    groups: ArrayLike | None = None,
    scoring: ArrayLike | tuple | ((...) -> Any) | Mapping | None = None,
    cv: int | BaseCrossValidator | Iterable | None = None,
    n_jobs: Int | None = None
)
```

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt_clf, X_train, y_train)

In [11]:
scores

{'fit_time': array([0.00592327, 0.00335264, 0.00409508, 0.00475717, 0.00348687]),
 'score_time': array([0.00066185, 0.00033617, 0.00043201, 0.00049996, 0.00025511]),
 'test_score': array([0.85480769, 0.85288462, 0.86525505, 0.85563041, 0.8719923 ])}

이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨있다. cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수도 있다.



교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

In [12]:
np.mean(scores['test_score'])

0.8601140149552083

교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다. 

#### cv 매개변수

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 앞서 우리는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없다. 하지만 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
stfold = StratifiedKFold(n_splits=8)
scores = cross_validate(dt_clf, X_train, y_train, cv=stfold)
# scores = cross_validate(dt_clf, X, y, cv=stfold)
print(np.mean(scores['test_score']))

0.8660750266682471


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다.

In [15]:
stfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=50)
scores = cross_validate(dt_clf, X_train, y_train, cv=stfold)
# scores = cross_validate(dt_clf, X, y, cv=stfold)
print(np.mean(scores['test_score']))

0.8643436805383166


## - 하이퍼파라미터 튜닝

하이퍼파라미터는 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터이다. 사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다. 그럼 이런 하이퍼파라미터를 튜닝하는 작업은 어떻게 할까? 

1. 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. 
2. 그 다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다. 
   - 모델마다 적게는 1 ~ 2개에서, 많게는 5 ~ 6개의 매개변수를 제공한다.

매개변수를 찾아 튜닝에 사용할 수 있는 GridSearchCV 와 랜덤서치인 GridRandomSearchCV를 알아보자.

#### GridSearchCV

사이킷런에서 제공하는 그리드서치를 사용해보자. 사이킷런의 GridSearchCV 클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_validate() 함수를 호출할 필요가 없다. 

```python
class GridSearchCV(
    estimator: DecisionTreeClassifier,
    param_grid: Mapping | Sequence[dict],
    *,
    scoring: ArrayLike | tuple | ((...) -> Any) | Mapping | None = None,
    n_jobs: Int | None = None,
    refit: str | ((...) -> Any) | bool = True,
    cv: int | BaseCrossValidator | Iterable | None = None,
    verbose: Int = 0,
    pre_dispatch: str | int = "2*n_jobs",
    error_score: Float | str = ...,
    return_train_score: bool = False
)
```

기존 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아보자.

```
class DecisionTreeClassifier(
    *,
    criterion: Literal['gini', 'entropy', 'log_loss'] = "gini",
    splitter: Literal['best', 'random'] = "best",
    max_depth: Int | None = None,
    min_samples_split: float = 2,
    min_samples_leaf: float = 1,
    min_weight_fraction_leaf: Float = 0,
    max_features: float | Literal['auto', 'sqrt', 'log2'] | None = None,
    random_state: Int | RandomState | None = None,
    max_leaf_nodes: Int | None = None,
    min_impurity_decrease: Float = 0,
    class_weight: Mapping | str | Sequence[Mapping] | None = None,
```

여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도해보자. GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다. 일반 모델을 훈련하는 것처럼 gs 객체에 fit() 메서드를 호출한다. 그리드서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행한다. 

In [16]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' :  [0.0001, 0.0002, 0.0003, 0.0004, 0.0005] }

dt_clf = DecisionTreeClassifier(random_state=50)
gs = GridSearchCV(dt_clf, param_grid=params, n_jobs=-1)

gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=50), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

GridSearchCV의 cv 매개변수 기본값은 5이다. min_impurity_decrease 값 마다 5-폴드 교차 검증을 수행한다. 결국 5x5=25 개의 모델을 훈련하게 된다. 

GridSearchCV 클래스의 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는것이 좋다. 이 매개변수의 기본값은 1이다. -1로 지정하면 시스템에 있는 모든 코어를 사용한다.

사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다. 
 - GridSearchCV 객체의 bestestimator_ 속성: 최적의 모델 모델
 - GridSearchCV 객체의 best_params_ 속성: 최적의 파라미터 값

In [17]:
dt = gs.best_estimator_ # <--- DecisionTreeClass
print(dt.score(X_train, y_train))

0.9299595920723495


In [18]:
gs.best_params_

{'min_impurity_decrease': 0.0002}

0.0002이 가장 좋은 값으로 선택되었다.

각 매개변수에서 수행한 교차 검증의 평균 점수는 cvresults 속성의 'mean_test_score' 키에 저장되어 있다. 5번의 교차 검증으로 얻은 점수를 출력해보자.

In [19]:
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_min_impurity_decrease', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [20]:
gs.cv_results_['mean_test_score']

array([0.86934923, 0.86973273, 0.8683875 , 0.86511513, 0.86588325])

In [21]:
gs.cv_results_['params']

[{'min_impurity_decrease': 0.0001},
 {'min_impurity_decrease': 0.0002},
 {'min_impurity_decrease': 0.0003},
 {'min_impurity_decrease': 0.0004},
 {'min_impurity_decrease': 0.0005}]

위처럼 수동으로 고르는 것보다 넘파이의 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다. 그 다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

In [22]:
idx = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][idx]

{'min_impurity_decrease': 0.0002}

#### 규제 추가

조금 더 복잡한 매개 변수 조합을 탐색해보자. 결정 트리에서 min_impurity_decreas는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다. 여기에 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라보자.

In [23]:
params = {
    'min_impurity_decrease' :  np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : range(5, 20, 1),
    'min_samples_split' : range(2, 100, 5)
}

넘파이arang() 함수는 첫 번째 매개변수 값에서 시작하여 두 번쨰 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다. 배열의 총 원소는 9개이다. 파이썬 range 함수도 비슷하다. 하지만 이 함수는 정수만 사용할 수 있다. max_depth는 15개의 값을 만들고, min_sample_split은 10개의 값을 만든다. 

이 매개변수로 수행할 교차 검증 횟수는 9x15x10 = 1350개이다. 기본 5-폴드 교차검증을 수행하므로 만들어지는 6750개이다. 다음 그리드 서치를 실행해 보자.

In [24]:
dt_clf = DecisionTreeClassifier(random_state=50)

In [25]:
gs = GridSearchCV(dt_clf, param_grid=params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=50), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 5)})

최상의 매개변수 조합을 확인해 보자.

In [26]:
gs.best_params_

{'max_depth': 15, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}

In [27]:
np.max(gs.cv_results_['mean_test_score'])

0.8716578440808469

In [28]:
dt_clf2 = DecisionTreeClassifier(
    min_impurity_decrease=0.0001,
    max_depth=15,
    min_samples_split=2,
    random_state=50
)

gs = GridSearchCV(dt_clf2, param_grid=params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(max_depth=15,
                                              min_impurity_decrease=0.0001,
                                              random_state=50),
             n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 5)})

In [29]:
best_model = gs.best_estimator_
print("Train score :", best_model.score(X_test, y_test))

Train score : 0.8692307692307693


### - 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 때 **랜덤 서치** 를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다. 싸이파이의 확률 분포 클래스를 사용할 수 있다.

In [30]:
from scipy.stats import uniform, randint

uniform과 randint 클래스는 모두 주어진 범위에서 고르게 정수, 실수 값을 뽑는다. 이를 균등 분포에서 샘플링한다 라고 한다.

In [31]:
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 7, 9, 5, 4, 2, 5, 0, 4, 0])

이번엔 1000개를 샘플링해서 각 숫자의 개수를 세어보자.

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([112,  95,  99, 106,  99, 107,  93,  99,  93,  97]))

개수가 늘어나니 0에서 9까지의 숫자가 어느 정도 고르게 추출된 것을 볼 수 있다. 

uniform 클래스의 사용법도 동일하다. 0 ~ 1 사이에서 10개의 실수를 추출해 보자.

In [33]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.43690298, 0.59420143, 0.65513551, 0.23891545, 0.07279596,
       0.00593554, 0.68147821, 0.2275249 , 0.96643983, 0.87683035])

#### RandomizedSearchCV 

이제 탐색할 매개변수의 딕셔너리를 만들어 보자. 여기에서는 min_samples_leaf 매개변수를 탐색 대상에 추가하자. 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수이다. 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.

In [41]:
params = {
    'min_impurity_decrease' :  np.arange(0.0001, 0.001),
    'max_depth' : range(20, 50),
    'min_samples_split' : range(2, 25),
    'min_samples_leaf' : range(1, 25)
}

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [42]:
from sklearn.model_selection import RandomizedSearchCV
dt_cls = DecisionTreeClassifier(random_state=50)
gs = RandomizedSearchCV(dt_cls, params, n_iter=100, n_jobs=-1, random_state=50)

위 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다. 앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다. 먼저 최적의 매개변수 조합을 출력하자.

In [43]:
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=50),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(20, 50),
                                        'min_impurity_decrease': array([0.0001]),
                                        'min_samples_leaf': range(1, 25),
                                        'min_samples_split': range(2, 25)},
                   random_state=50)

In [44]:
gs.best_params_

{'min_samples_split': 23,
 'min_samples_leaf': 16,
 'min_impurity_decrease': 0.0001,
 'max_depth': 38}

In [45]:
np.max(gs.cv_results_['mean_test_score'])

0.8681948249056045

최적의 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_속성에 저장되어 있다. 이 모델을 최종으로 테스트 세트의 성능을 확인해 보자.

In [46]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=38, min_impurity_decrease=0.0001,
                       min_samples_leaf=16, min_samples_split=23,
                       random_state=50)

In [48]:
dt.score(X_test, y_test)

0.8684615384615385

#### splitter 매개변수를 사용해 보자.

In [86]:
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=50)
dt_cls = DecisionTreeClassifier(splitter='best',random_state=50)
gs = RandomizedSearchCV(dt_cls, params, cv=splitter, n_iter=100, n_jobs=-1, random_state=50)
gs.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=50, shuffle=True),
                   estimator=DecisionTreeClassifier(random_state=50),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(20, 50),
                                        'min_impurity_decrease': array([0.0001]),
                                        'min_samples_leaf': range(1, 25),
                                        'min_samples_split': range(2, 25)},
                   random_state=50)

In [87]:
gs.best_params_

{'min_samples_split': 14,
 'min_samples_leaf': 7,
 'min_impurity_decrease': 0.0001,
 'max_depth': 29}

In [88]:
np.max(gs.cv_results_['mean_test_score'])

0.8662642655995256

In [89]:
dt = gs.best_estimator_

In [90]:
dt.score(X_test, y_test)

0.8592307692307692

In [91]:
# 모델 추출
import joblib

# joblib을 사용하여 모델 저장
model_filename = './model/wine.joblib'
joblib.dump(gs, model_filename)

['./model/wine.joblib']